In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import re
import numpy as np

In [ ]:
# They have ea, 100g, kg, 100ml, ltr, 75cl, 

In [46]:
df = pd.read_csv("Sainsburys.csv")

In [57]:
df.head(10)

,Name,Price,Nectar price,Category,Subcategory,Standardised price per unit,Unit_each,Unit_kg,Unit_litre,Year,Month,Day
0,Sainsbury's Fairtrade Bananas Loose,0.90,NaN,fresh_food,fruits,0.90,0.0,1.0,0.0,2024,11,19
1,Sainsbury's Fairtrade Bananas x5,0.78,NaN,fresh_food,fruits,0.16,1.0,0.0,0.0,2024,11,19
2,Sainsbury's Red Seedless Grapes 500g,2.00,NaN,fresh_food,fruits,4.00,0.0,1.0,0.0,2024,11,19
3,"Sainsbury's Easy Peelers, Taste the Difference...",2.00,NaN,fresh_food,fruits,3.33,0.0,1.0,0.0,2024,11,19
4,Sainsbury's Blueberries 150g,2.00,NaN,fresh_food,fruits,13.33,0.0,1.0,0.0,2024,11,19
5,Sainsbury's Strawberries 250g,2.50,NaN,fresh_food,fruits,10.00,0.0,1.0,0.0,2024,11,19
6,Sainsbury's Fairtrade Bananas x8,1.25,NaN,fresh_food,fruits,0.16,1.0,0.0,0.0,2024,11,19
7,Sainsbury's White Seedless Grapes 500g,2.00,NaN,fresh_food,fruits,4.00,0.0,1.0,0.0,2024,11,19
8,Sainsbury's Raspberries 150g,1.90,NaN,fresh_food,fruits,12.67,0.0,1.0,0.0,2024,11,19
9,Sainsbury's Blueberries 300g,3.40,2.8,fresh_food,fruits,11.33,0.0,1.0,0.0,2024,11,19


In [48]:
def clean_price(value):
    """
    Cleans the Price column by:
    - Handling pence (e.g., '90p / kg' or '90p') by removing 'p' and dividing by 100.
    - Handling pounds (e.g., '£2.50') by removing '£' and converting to float.
    - Extracting only the numeric value before '/' if such a case exists.
    """
    if isinstance(value, str):  # Ensure the value is a string
        if '/' in value:  # Special case: '90p / kg'
            value = value.split('/')[0].strip()  # Take only the part before '/'
        
        if 'p' in value:  # Case for pence
            return float(value.replace('p', '').strip()) / 100
        elif '£' in value:  # Case for pounds
            return float(value.replace('£', '').strip())
    return np.nan  # Return NaN for invalid or missing values


In [51]:
# Apply the cleaning function to the 'Nectar price' column
df['Nectar price'] = df['Nectar price'].apply(clean_price)



In [49]:
# Apply the cleaning function to the 'Nectar price' column
df['Price'] = df['Price'].apply(clean_price)

In [56]:
# Convert 'Date' column to datetime format (this automatically handles the conversion to day, month, and year)
df['Date'] = pd.to_datetime(df['Date'])

# Split the 'Date' column into individual components: Year, Month, Day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
# Drop the original 'Date' column if it's no longer needed
df = df.drop(columns=['Date'])

In [53]:
def standardize_price_per_unit(price_per_unit):
    if isinstance(price_per_unit, str):  # Ensure the value is a string
        
        # Split into price and unit if 'per' is in the string
        if '/' in price_per_unit:
            try:
                price_value, unit = price_per_unit.split(' / ')  # Split into price and unit
                
                # If there's no decimal point, we assume it's in whole pence (e.g., '15.7p' -> '0.157')
                if 'p' in price_value:
                    if '.' not in price_value:
                        price_value = f"{float(price_value.replace('p', '').strip()) / 100:.3f}"  # Convert pence to pound
                    else:
                        price_value = price_value.replace('p', '')  # Just remove 'p' (since we are handling it as pence already)
                
                price_value = float(price_value.replace('£', '').strip())  # Convert price to float and remove '£'
                
                # Unit conversions based on the specific units
                if '100g' in unit:  # Convert 100g to kg
                    price_value = price_value * 10  # 100g is 0.1kg, so we multiply price by 10
                    unit = 'kg'
                elif '10g' in unit:
                    price_value = price_value * 100
                    unit = 'kg'
                elif 'kg' in unit:  # No conversion needed
                    unit = 'kg'
                elif '100ml' in unit:  # Convert 100ml to litre
                    price_value = price_value * 10  # 100ml is 0.1l, so we multiply price by 10
                    unit = 'litre'
                elif 'ltr' in unit:  # No conversion needed
                    unit = 'litre'
                elif '75cl' in unit:  # Convert cl to litre (e.g., 75cl to 0.75l)
                    price_value = price_value / 10  # 75cl = 0.75l, so divide by 10
                    unit = 'litre'
                elif 'ea' in unit:  # Handle 'each' (e.g., '5.20 each')
                    unit = 'each'
                
                return price_value, unit  # Return the price and unit as a tuple

            except Exception as e:
                # In case of any error (e.g., splitting or parsing), return NaN values
                return np.nan, np.nan
        else:
            return np.nan, np.nan  # Return NaN values if the format doesn't match 'per'
    else:
        return np.nan, np.nan  # Return NaN if the value is not a string


In [54]:
# Apply the function to the dataframe
df[['Standardised price per unit', 'Unit']] = df['Price per Unit'].apply(
    lambda x: pd.Series(standardize_price_per_unit(x))
)

# Filter out invalid unit values (only keep valid units)
df = df[df['Unit'].isin(['kg', 'litre', 'each'])]

# One-hot encode the 'Unit' column
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')  # Avoid errors for unknown units
unit_encoded = encoder.fit_transform(df[['Unit']])

# Create new columns based on one-hot encoding
unit_columns = encoder.get_feature_names_out(['Unit'])
df[unit_columns] = unit_encoded

# Drop the original 'Unit' and 'Price per Unit' columns
df.drop(columns=['Unit', 'Price per Unit'], inplace=True)

# Display the cleaned dataframe
print(df.head())

                                                Name  Price  Nectar price  \
0                Sainsbury's Fairtrade Bananas Loose   0.90           NaN   
1                   Sainsbury's Fairtrade Bananas x5   0.78           NaN   
2               Sainsbury's Red Seedless Grapes 500g   2.00           NaN   
3  Sainsbury's Easy Peelers, Taste the Difference...   2.00           NaN   
4                       Sainsbury's Blueberries 150g   2.00           NaN   

     Category Subcategory        Date  Standardised price per unit  Unit_each  \
0  fresh_food      fruits  2024-11-19                         0.90        0.0   
1  fresh_food      fruits  2024-11-19                         0.16        1.0   
2  fresh_food      fruits  2024-11-19                         4.00        0.0   
3  fresh_food      fruits  2024-11-19                         3.33        0.0   
4  fresh_food      fruits  2024-11-19                        13.33        0.0   

   Unit_kg  Unit_litre  
0      1.0         0.0  


c:\Users\Entwan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
